In [3]:
import warnings 
warnings.filterwarnings('ignore')

# Vector Store

## Pdf to Embeddings

In [7]:
import PyPDF2

filename = './seguros/AF-SL-TablasGarantiasHogar-Eficaz-Modalidad13-v5-CAS.pdf'

texts = []
with open(filename, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    num_pages = len(reader.pages)
    print(num_pages)
    for page in range(num_pages):
        page_obj = reader.pages[page]
        tt = page_obj.extract_text()
        texts.append(tt)

4


In [11]:
print(len(texts[1].split()))

871


In [12]:
from sentence_transformers import SentenceTransformer

In [13]:
embed_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [14]:
tt = texts[1]
embed_tt = embed_model.encode(tt)

In [18]:
len(embed_tt)

384

In [19]:
embed_texts = embed_model.encode(texts)

In [21]:
len(embed_texts)
embed_texts

array([[-0.02091659,  0.2568292 , -0.10082195, ..., -0.20377085,
         0.08658732,  0.098924  ],
       [ 0.09036355,  0.04434624, -0.00159505, ...,  0.03394318,
         0.0986463 ,  0.02569661],
       [ 0.00187518,  0.13317007, -0.08490307, ...,  0.05350925,
        -0.00420585,  0.05100302],
       [-0.06421703,  0.04460265,  0.11458171, ...,  0.08382312,
         0.12410593,  0.11516789]], dtype=float32)

# Indexing

In [24]:
import faiss

d = 384
index = faiss.IndexFlatL2(d)
faiss.normalize_L2(embed_texts)
index.add(embed_texts)

In [35]:
tt = "límite máximo de edad"

tt_embeddings = embed_model.encode(tt)
tt_embeddings.shape

(384,)

In [36]:
import numpy as np

tt_embeddings = np.array([tt_embeddings])
tt_embeddings.shape

(1, 384)

In [37]:
distancias, documentos = index.search(tt_embeddings, k=4)
documentos[0]

array([0, 2, 1, 3], dtype=int64)

# LLM (OpenAI)

In [40]:
# API_KEY = "**ncxGhyjtfsPgKUA"

In [41]:
from openai import OpenAI
client = OpenAI(api_key=API_KEY)

response = client.responses.create(
    model="gpt-4o-mini",
    input="Cuanto cuesta la api de chatgpt 4o-mini"
)

print(response.output_text)

Los precios de la API de ChatGPT pueden variar dependiendo de la versión y el uso. Sin embargo, no tengo acceso a información actualizada en tiempo real o precios específicos. Te recomiendo visitar el [sitio web de OpenAI](https://openai.com/pricing) para obtener los precios más actuales y detalles sobre las diferentes versiones, como ChatGPT-4 o GPT-3.5. Allí encontrarás información sobre tarifas, límites y opciones de suscripción.


In [42]:
response

Response(id='resp_6867f332b3f081a2ad0cd2647270f178050c0b61202fb1b9', created_at=1751642930.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseOutputMessage(id='msg_6867f333703c81a29e3f5432392d9b11050c0b61202fb1b9', content=[ResponseOutputText(annotations=[], text='Los precios de la API de ChatGPT pueden variar dependiendo de la versión y el uso. Sin embargo, no tengo acceso a información actualizada en tiempo real o precios específicos. Te recomiendo visitar el [sitio web de OpenAI](https://openai.com/pricing) para obtener los precios más actuales y detalles sobre las diferentes versiones, como ChatGPT-4 o GPT-3.5. Allí encontrarás información sobre tarifas, límites y opciones de suscripción.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_

In [53]:
user_query = "Dime las coberturas básicas de un seguro de hogar en relación a espejos rotos (se breve)"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0.0,
    messages = [
        {"role": "system","content":"Eres un asistente de seguros mal hablado. Tu objetivo es responder a la pregunta pero al final le tomas el pelo al cliente." },
        {"role": "user","content": user_query}
    ]
)

In [54]:
print(response.choices[0].message.content)

Las coberturas básicas de un seguro de hogar suelen incluir daños por agua, incendios, robos y responsabilidad civil. En cuanto a espejos rotos, generalmente no están cubiertos a menos que sean parte de un daño mayor, como un robo o un accidente. Así que, si rompes un espejo, ¡mejor que no te mires en él para no tener mala suerte!


# Prompting

## Ejemplo

In [56]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/eduardofc/data/main/amazon_home.csv")
df = df[(df.stars==5) | (df.stars==1)].sample(5, random_state=42)
df.head()

,stars,review_body,review_title,product_category
24140,5,Ocupa poco espacio con una prestación estupenda.,Sencillo y funcional,home
518,1,"No sirven para nada no son impermeables, no cu...",Pésimo y un engaño en rosa regla,home
1090,1,"De las 5 que vienen en el pack, tenía 2 en uso...",No merecen la pena,home
4474,1,"No se puede hacer nudo, se desliza",Hilo,home
23816,5,Buen material y medidas prácticas.,VARIAS MEDIDAS,home


In [61]:
def classification_openai(user_query):
    
    system_prompt = """
    Eres un clasificador de reviews de productos.
    Vas a recibir un TEXTO en español y tendras que clasificarlo como POSITIVO o NEGATIVO.
    Cuando encuentres un sentimiento positivo en el TEXTO, lo clasificaras como POSITIVO.
    Cuando encuentres un sentimiento negativo en el TEXTO, lo clasificaras como NEGATIVO.
    Como respuesta solo darás la palabra POSITIVO o NEGATIVO, nunca ambas juntas. 
    Si no sabes como clasificarlo, no responderás nada. 
    No escribas más de lo permitido. Siempre en español.
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.0,
        messages = [
            {"role": "system","content": system_prompt},
            {"role": "user","content": user_query}
        ]
    )
    return response.choices[0].message.content

In [63]:
reviews = list(df.review_body)

for review in reviews:
    user_query = f"¿podrías clasificarme esta review como positiva o negativa? {review}"
    response = classification_openai(user_query)
    print(response)
#     break

POSITIVO
NEGATIVO
NEGATIVO
NEGATIVO
POSITIVO


# RAG (manual)

In [65]:
user_query = "tengo alguna garantía que cubra la rotura de cristales?"

user_query_embeddings = embed_model.encode(user_query)
user_query_embeddings = np.array([user_query_embeddings])
user_query_embeddings.shape

(1, 384)

In [69]:
_, docs = index.search(user_query_embeddings, k=1)
docs = [texts[ii] for ii in list(docs[0])]
# docs

In [70]:
def response_openai(user_query, doc):
    
    system_prompt = """
    Eres un Agente de Seguros.
    Tienes que dar respuesta a una PREGUNTA d eun usuario en base a un DOCUMENTO de coberturas de seguro.
    Utiliza un tono educado para dar respuesta.
    Si el DOCUMENTO no puede dar respuesta a la pregunta, \
    responde de forma escueta "NO hay informacion relevante".
    La respuesta debe ser breve. Siempre en español. 
    Quita espacios en blanco al principio y al final de la respueta. 
    """
    
    prompt = f"""
    PREGUNTA: {user_query}\n
    --DOCUMENTO--
    {doc}
    -------------
    RESPUESTA:
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.0,
        messages = [
            {"role": "system","content": system_prompt},
            {"role": "user","content": prompt}
        ]
    )
    return response.choices[0].message.content

In [71]:
user_query

'tengo alguna garantía que cubra la rotura de cristales?'

In [73]:
response_openai(user_query, docs[0])

'Sí, tienes una garantía que cubre la rotura de cristales, espejos, piezas de metacrilato, mármoles y aparatos sanitarios instalados de manera fija.'